In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install bitsandbytes
!pip install transformers
!pip install datasets
!pip install groqflow
!pip install accelerate
!pip install groq
!pip install langchain
!pip install langchain-community
!pip install wolframalpha
!pip install -qU langchain-groq
!pip install -qU langchain-anthropic
!pip install -qU langchain_mistralai

In [ ]:
from datasets import load_dataset

# Load the gsm8k dataset
dataset = load_dataset("openai/gsm8k",'main')

# Display the first few rows
print(dataset)
print(dataset['train'][0])

In [ ]:
from langchain_mistralai import ChatMistralAI
from kaggle_secrets import UserSecretsClient
from langchain_groq import ChatGroq
from langchain.agents import load_tools
from langchain.agents import AgentExecutor,create_react_agent,initialize_agent,AgentType,Tool
from langchain_anthropic import ChatAnthropic
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_retries=2,
    api_key="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"
)



tools = load_tools(["llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent_type=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,handle_parsing_errors=True, verbose=True)
messages = [
    (
        "system",
        "You are a helpful assistant that does simple maths calculations. You have access to tools like caculators and wolfram alpha?"
        "If you need to use a tool, state the action clearly first, then wait for the result before providing the final answer. "
        "Do not provide the final answer until you have received all necessary observations from the tools.",
    ),
    ("human", "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"),
]
result = agent.run(messages)


In [ ]:
import re
from io import StringIO
from transformers import TextIteratorStreamer
import sys
import re
import io
def run_and_capture(prompt, react_agent):
    # Create a string buffer to capture the output
    captured_output = io.StringIO()
    
    # Save the original stdout
    original_stdout = sys.stdout
    
    try:
        # Redirect stdout to the buffer
        sys.stdout = captured_output

        # Run the agent with the provided prompt
        result = react_agent.run(prompt)
        
        # Ensure the captured output is written to original stdout (console)
        original_stdout.write(captured_output.getvalue())

    finally:
        # Reset stdout to its original state
        sys.stdout = original_stdout
    
    # Get the captured verbose output
    verbose_output = captured_output.getvalue()
    return verbose_output
def extract_final_answer_2(thought_process):
    # Split the text by lines to process each line separately
    lines = thought_process.strip().splitlines()
    
    final_observation = None  # To store the last valid observation before the final answer
    found_final_answer = False
    
    # Traverse through the lines to identify and extract the relevant parts
    for line in lines:
        # Check if the line is an observation
        if line.startswith("Observation:"):
            # Extract the answer from the observation
            observation_match = re.search(r"Answer:\s*([\d\.]+)", line)
            if observation_match:
                final_observation = observation_match.group(1)  # Update the final observation
            
        # Check if the line contains "Thought:Final Answer:"
        if "Final Answer:" in line:
            
            break  # Stop processing further since we found the final answer thought

    # Return the final observation found before the final answer
    return final_observation
def extract_final_answer(thought_process):
    # Split the text by lines to process each line separately
    lines = thought_process.strip().splitlines()
    
    # Traverse through the lines to identify and extract the relevant final answer
    for line in lines:
        # Check if the line contains "Final Answer:"
        if "Final Answer:" in line:
            # Extract only the number from the final answer line
            final_answer_match = re.search(r"Final Answer:.*?([\d\.]+)", line)
            if final_answer_match:
                return final_answer_match.group(1).strip()
    
    # Return None if no final answer is found
    return None
def extract_final_answer_from_answer(output):
    # Use a regular expression to capture the answer after "####"
    match = re.search(r"####\s*(\d+)", output)
    
    if match:
        return match.group(1)  # Return the captured answer
    else:
        return None  # Return None if the answer is not found

In [ ]:

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_retries=2,
    api_key="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"
)
tools = load_tools(["llm-math","wolfram-alpha"],wolfram_alpha_appid="4L7YU5-AU95ELH8ER", llm=llm)
agent = initialize_agent(tools, llm, agent_type=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,handle_parsing_errors=True, verbose=True,max_iterations=3)
num_correct=0
invalid_extractions=0
total=0
for idx, entry in enumerate(dataset["train"]):
    question = entry["question"]
    answer = entry["answer"]
    correct_answer= extract_final_answer_from_answer(answer)
    total+=1
    messages = [
        (
            "system",
            "You are a helpful assistant that does simple maths calculations. You have access to tools like caculators and wolfram alpha?"
            "If you need to use a tool, state the action clearly first, then wait for the result before providing the final answer. "
            "Do not provide the final answer until you have received all necessary observations from the tools."
            "Please ensure the final answer is only a single number without any units or statements.",
        ),
        ("human", question),
    ]
    try:
        result = run_and_capture(messages,agent)
        generated_answer=extract_final_answer(result)
        if correct_answer is not None and generated_answer is not None:
            correct_answer_int = int(float(correct_answer))
            generated_answer_int = int(float(generated_answer))
                # Compare the integer values
            if correct_answer_int == generated_answer_int:
                num_correct+=1
                print("Yay")
        else:
            invalid_extractions+=1
            print("----------------------------------------------------")
    except:
        invalid_extractions+=1
        result=""
    if(idx>=100):
        break

accuracy=(num_correct/(total-invalid_extractions))*100
print(f"Accuracy: {accuracy}")
print(f"Invalid extractions {invalid_extractions}")

In [ ]:

llm = ChatGroq(
    model="gemma2-9b-it",
    temperature=0,
    max_retries=2,
    api_key="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"
)
tools = load_tools(["llm-math","wolfram-alpha"],wolfram_alpha_appid="4L7YU5-AU95ELH8ER", llm=llm)
agent = initialize_agent(tools, llm, agent_type=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,handle_parsing_errors=True, verbose=True)
num_correct=0
invalid_extractions=0
total=0
for idx, entry in enumerate(dataset["train"]):
    question = entry["question"]
    answer = entry["answer"]
    correct_answer= extract_final_answer_from_answer(answer)
    total+=1
    messages = [
        (
            "system",
            "You are a helpful assistant that does simple maths calculations. You have access to tools like caculators and wolfram alpha?"
            "If you need to use a tool, state the action clearly first, then wait for the result before providing the final answer. "
            "Do not provide the final answer until you have received all necessary observations from the tools."
            "Please ensure the final answer is only a single number without any units or statements.",
        ),
        ("human", question),
    ]
    try:
        result = run_and_capture(messages,agent)
        generated_answer=extract_final_answer(result)
        if correct_answer is not None and generated_answer is not None:
            correct_answer_int = int(float(correct_answer))
            generated_answer_int = int(float(generated_answer))
                # Compare the integer values
            if correct_answer_int == generated_answer_int:
                print("Yay")
                num_correct+=1
        else:
            invalid_extractions+=1
            print("----------------------------------------------------")
    except Exception as e:
        print(e)
        invalid_extractions+=1
        result=""
    if(idx>=100):
        break

accuracy=(num_correct/(total-invalid_extractions))*100
print(f"Accuracy: {accuracy}")
print(f"Invalid extractions {invalid_extractions}")

In [ ]:

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2,
    api_key="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"
)
tools = load_tools(["llm-math","wolfram-alpha"],wolfram_alpha_appid="4L7YU5-AU95ELH8ER", llm=llm)
agent = initialize_agent(tools, llm, agent_type=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,handle_parsing_errors=True, verbose=True)
num_correct=0
invalid_extractions=0
total=0
for idx, entry in enumerate(dataset["train"]):
    question = entry["question"]
    answer = entry["answer"]
    correct_answer= extract_final_answer_from_answer(answer)
    total+=1
    messages = [
        (
            "system",
            "You are a helpful assistant that does simple maths calculations. You have access to tools like caculators and wolfram alpha?"
            "If you need to use a tool, state the action clearly first, then wait for the result before providing the final answer. "
            "Do not provide the final answer until you have received all necessary observations from the tools."
            "Please ensure the final answer is only a single number without any units or statements.",
        ),
        ("human", question),
    ]
    try:
        result = run_and_capture(messages,agent)
        generated_answer=extract_final_answer_2(result)
        if correct_answer is not None and generated_answer is not None:
            correct_answer_int = int(float(correct_answer))
            generated_answer_int = int(float(generated_answer))
                # Compare the integer values
            if correct_answer_int == generated_answer_int:
                print("Yay")
                num_correct+=1
        else:
            invalid_extractions+=1
            print("----------------------------------------------------")
    except:
        invalid_extractions+=1
        result=""
    if(idx>=100):
        break

accuracy=(num_correct/total)*100
print(f"Accuracy: {accuracy}")
print(f"Invalid extractions {invalid_extractions}")